# 예측 모델

## 데이터 불러오기

In [1]:
import connectorx as cx
import pandas as pd
import numpy as np
import time
import warnings
import math
import os

import pymysql
from sqlalchemy import create_engine
import pandas as pd
from sqlalchemy.types import VARCHAR, Integer
import datetime
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

# bontemp 실험환경 데이터 불러오기
user = "direct"
password = "Direct999"
host = "server5.marketingtool.co.kr"
port = 60001
db = "direct"
uri = f'''mysql://{user}:{password}@{host}:{port}/{db}'''
table = "lgcns_miecsm_daily_output_projection"


item_user = "sales"
item_password = "sales123"
item_host = "server5.marketingtool.co.kr"
item_port = 23061
item_db = "lgcns_miecsm"
item_uri = f'''mysql://{item_user}:{item_password}@{item_host}:{item_port}/{item_db}'''
item_table = "item_master"

In [2]:
delivery_data0 = pd.read_parquet("/home/jupyter/ECSM/3_lg/01_Projection/projoutput2401_2501.parquet")
delivery_data1 = pd.read_parquet("/home/jupyter/ECSM/3_lg/01_Projection/projoutput_2502_251220.parquet")

In [3]:
from datetime import datetime, timedelta

sdate = "2025-12-21"
edate = "2025-12-28"

In [4]:
# 예측 대상 마켓만 불러오기
query = f"""
SELECT * FROM {db}.{table}
where 1=1
and rdate between "{sdate}" and "{edate}"
"""
delivery_data2 = cx.read_sql(conn = uri, query = query)

In [5]:
print(delivery_data0.shape)
print(delivery_data1.shape)
delivery_data = pd.concat([delivery_data0, delivery_data1] , axis = 0)
print(delivery_data.shape)
delivery_data = pd.concat([delivery_data, delivery_data2] , axis = 0)
print(delivery_data.shape)

del delivery_data0, delivery_data1, delivery_data2

(83592657, 21)
(91490763, 21)
(175083420, 21)
(177586834, 21)


In [6]:
# 카테고리
c_table = "category_master_"
query = f"""
SELECT distinct cat_id, cat_name
FROM {db}.{c_table}
where 1=1
and user_id = "lgcns_miecsm"
"""
category_master = cx.read_sql(conn = uri, query = query)
category_master.rename(columns = {"cat_id":"category"}, inplace = True)

# 브랜드
item_user = "sales"
item_password = "sales123"
item_host = "server5.marketingtool.co.kr"
item_port = 23061
item_db = "lgcns_miecsm"
item_uri = f'''mysql://{item_user}:{item_password}@{item_host}:{item_port}/{item_db}'''
item_table = "item_master"

query = f"""
SELECT std_item_cd, std_mnfr FROM {item_db}.{item_table}
"""
item_master = cx.read_sql(conn = item_uri, query = query)

In [7]:
df1 = pd.merge(delivery_data,category_master, on = ["category"], how = "left")
 
df2 = pd.merge(df1, item_master, on = ["std_item_cd"], how = "left")

In [8]:
df2['cat_name']=np.where(df2['cat_name']=="김치냉장고","냉장고",df2['cat_name'])
df2['cat_name']=np.where(df2['cat_name']=="워시타워","세탁기",df2['cat_name'])
df2['cat_name']=np.where(df2['cat_name']=="건조기","세탁기",df2['cat_name'])
df2['cat_name']=np.where(df2['cat_name']=="로봇청소기","무선청소기",df2['cat_name'])

In [9]:
# 브랜드 구분
df2['brand'] = np.where(df2['std_mnfr']=="LG전자", df2['std_mnfr'], np.where(df2['std_mnfr']=="삼성전자", df2['std_mnfr'],"etc"))
df3 = df2.groupby(['rdate','mall','cat_name','brand'], as_index=False)['final_sales_unit'].sum()

## 미수집 시 데이터 생성

In [10]:
ddf = df3.query('(rdate>="2025-12-28")&(mall!="south.kr.smartstore")')[['rdate','mall','cat_name','brand','final_sales_unit']]
ddf['final_sales_unit'] = 0
ddf['mall'] = "south.kr.smartstore"

ddf = ddf.drop_duplicates()

In [11]:
df4 = df3.query('~(rdate>="2025-12-28"&mall=="south.kr.smartstore")')

df4 = pd.concat([df4, ddf] , axis = 0)

In [12]:
# 요일 구분
df4['week'] = df4['rdate'].dt.day_name()
df4.reset_index(drop=True,inplace=True)

## 추가 데이터 불러오기(네이버 데이터랩)

### 1. 네이버 데이터랩

In [13]:
# 정보
n_user = "shchoo"
n_password = "shchoo000"
n_host = "marketingtool.co.kr"
n_port = 23327
n_db = "trend"
n_uri = f'''mysql://{n_user}:{n_password}@{n_host}:{n_port}/{n_db}'''

In [14]:
# 데이터
table1 = "naver_keyword_trend"
query = f"""
SELECT * FROM {n_db}.{table1}
where male_yn = "N"
and female_yn = "N"
and age_35_older_yn = "N"
and age_40_older_yn = "N"
and e_date = "{edate}"
"""
master1 = cx.read_sql(conn = n_uri, query = query)

master1 = master1[['seq','keyword','s_date','e_date']]
master1.rename(columns = {"seq":"search_seq"}, inplace = True)
master1 = master1.drop_duplicates(subset=['keyword','s_date','e_date'], keep="first")
xx = tuple(master1['search_seq'].unique())

# 검색어 트렌드 데이터
table2 = "naver_keyword_trend_data"
query = f"""
SELECT * FROM {n_db}.{table2}
where search_seq in {xx}
"""
data1 = cx.read_sql(conn = n_uri, query = query)

# 데이터 조인
datalab = pd.merge(data1[['search_seq','period','value']], master1, on="search_seq", how="left")

# 전처리
datalab.loc[datalab["keyword"] == "11번가", "keyword"] = "south.kr.11st"
datalab.loc[datalab["keyword"] == "G마켓", "keyword"] = "south.kr.gmarket"
datalab.loc[datalab["keyword"] == "옥션", "keyword"] = "south.kr.auction"
datalab.loc[datalab["keyword"] == "네이버쇼핑", "keyword"] = "south.kr.smartstore"
datalab.loc[datalab["keyword"] == "ssg닷컴", "keyword"] = "south.kr.ssgdotcom"
datalab.loc[datalab["keyword"] == "하이마트", "keyword"] = "south.kr.himart"
datalab.loc[datalab["keyword"] == "청소기", "keyword"] = "무선청소기"
datalab.loc[datalab["keyword"] == "인덕션", "keyword"] = "전기레인지"
# value 값 변경
datalab = datalab.astype({'value':'float64'})
datalab['cat_name'] = datalab['keyword']

# 컬럼명
datalab.rename(columns = {"keyword":"mall", "period":"rdate"}, inplace = True)

datalab0=datalab.copy()

datalab0.query('value==0')

,rdate,value,search_seq,mall,s_date,e_date,cat_name


In [15]:
datalab.head()

,rdate,value,search_seq,mall,s_date,e_date,cat_name


In [16]:
df5 = pd.merge(df4, datalab0[['rdate','mall','value']], on=["rdate","mall"], how="left")
df5.rename(columns = {"value":"mall_naverT"}, inplace = True)
df5 = pd.merge(df5, datalab0[['rdate','cat_name','value']], on=["rdate","cat_name"], how="left")
df5.rename(columns = {"value":"category_naverT"}, inplace = True)

#df5 = pd.merge(df5, datalab1[['rdate','mall','value']], on=["rdate","mall"], how="left")
#df5.rename(columns = {"value":"mall_naverM"}, inplace = True)
#df5 = pd.merge(df5, datalab1[['rdate','cat_name','value']], on=["rdate","cat_name"], how="left")
#df5.rename(columns = {"value":"category_naverM"}, inplace = True)

#df5 = pd.merge(df5, datalab2[['rdate','mall','value']], on=["rdate","mall"], how="left")
#df5.rename(columns = {"value":"mall_naverF"}, inplace = True)
#df5 = pd.merge(df5, datalab2[['rdate','cat_name','value']], on=["rdate","cat_name"], how="left")
#df5.rename(columns = {"value":"category_naverF"}, inplace = True)

### 2. 쇼핑인사이트

In [17]:
# 데이터
table1 = "naver_keyword_statistics"
query = f"""
SELECT * FROM {n_db}.{table1}
where male_yn = "Y"
and female_yn = "Y"
and pc_yn = "Y"
and age_30_yn = "Y"
and age_40_yn = "Y"
and s_date in ("2024-01-01","2024-11-27") 
and e_date = "{edate}"
"""
master2 = cx.read_sql(conn = n_uri, query = query)

master2 = master2[['seq','keyword','s_date','e_date']]
master2.rename(columns = {"seq":"search_seq"}, inplace = True)
master2 = master2.drop_duplicates(subset=['keyword','s_date','e_date'], keep="first")
xx = tuple(master2['search_seq'].unique())

# 검색어 트렌드 데이터
table2 = "naver_keyword_statistics_click_data"
query = f"""
SELECT * FROM {n_db}.{table2}
where search_seq in {xx}
"""
data2 = cx.read_sql(conn = n_uri, query = query)

# 데이터 조인
datalab = pd.merge(data2[['search_seq','period','value']], master2, on="search_seq", how="left")

# 전처리
datalab.loc[datalab["keyword"] == "청소기", "keyword"] = "무선청소기"
datalab.loc[datalab["keyword"] == "인덕션", "keyword"] = "전기레인지"
datalab = datalab.astype({'value':'float64'})
datalab.rename(columns = {"keyword":"cat_name", "period":"rdate"}, inplace = True)

datalab3=datalab.copy()

datalab3.query('value==0')

RuntimeError: MySqlError { ERROR 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near ')
 LIMIT 1' at line 2 }

In [ ]:
df6 = pd.merge(df5, datalab3[['rdate','cat_name','value']], on=["rdate","cat_name"], how="left")
df6.rename(columns = {"value":"category2_naverT"}, inplace = True)

## 성장률 데이터 활용

In [18]:
df6=df5.copy()

In [19]:
growth = df2.copy()

In [20]:
del df1, df2, df3, df4, df5

In [21]:
# 예측 마켓은 제외
growth1 = growth.query('~(mall=="south.kr.ohou"|mall=="south.kr.himart")').groupby(['rdate','mall','cat_name','brand'], as_index=False)['final_sales_unit'].sum()

growth1 = growth1.sort_values(['cat_name','brand','mall','rdate'],ascending=True)
growth1['sales_unit_lag'] = growth1['final_sales_unit'].shift(1)
growth1['cat_name_lag'] = growth1['cat_name'].shift(1)
growth1['brand_lag'] = growth1['brand'].shift(1)
growth1['mall_lag'] = growth1['mall'].shift(1)

growth1['total_ratio'] = np.where(
            (growth1['cat_name_lag'] == growth1['cat_name'])&
        (growth1['brand_lag'] == growth1['brand'])&
        (growth1['mall_lag'] == growth1['mall'])    
        ,growth1['final_sales_unit']/growth1['sales_unit_lag']
        ,1)

growth1 = growth1.fillna(0)
growth1['total_ratio'] = np.where((growth1['sales_unit_lag']==0)&(growth1['final_sales_unit']!=0), growth1['final_sales_unit'], growth1['total_ratio'])
growth1['total_ratio'] = np.where(growth1['total_ratio'] == 0, 0, np.where(growth1['total_ratio'] < 0.28, 0.28, np.where(growth1['total_ratio'] > 3.5, 3.5, growth1['total_ratio'])))

eleven = growth1.query('mall=="south.kr.11st"')[['rdate','cat_name','brand','total_ratio']]
eleven.rename(columns = {"total_ratio":"11st_ratio"}, inplace = True)

smartstore = growth1.query('mall=="south.kr.smartstore"')[['rdate','cat_name','brand','total_ratio']]
smartstore.rename(columns = {"total_ratio":"smartstore_ratio"}, inplace = True)

gmarket = growth1.query('mall=="south.kr.gmarket"')[['rdate','cat_name','brand','total_ratio']]
gmarket.rename(columns = {"total_ratio":"gmarket_ratio"}, inplace = True)

ssgdotcom = growth1.query('mall=="south.kr.ssgdotcom"')[['rdate','cat_name','brand','total_ratio']]
ssgdotcom.rename(columns = {"total_ratio":"ssgdotcom_ratio"}, inplace = True)

auction = growth1.query('mall=="south.kr.auction"')[['rdate','cat_name','brand','total_ratio']]
auction.rename(columns = {"total_ratio":"auction_ratio"}, inplace = True)

#tmon = growth1.query('mall=="south.kr.tmon"')[['rdate','cat_name','brand','total_ratio']]
#tmon.rename(columns = {"total_ratio":"tmon_ratio"}, inplace = True)

#wemake = growth1.query('mall=="south.kr.wemakeprice"')[['rdate','cat_name','brand','total_ratio']]
#wemake.rename(columns = {"total_ratio":"wemake_ratio"}, inplace = True)

In [22]:
# 예측 마켓은 제외
growth2 = growth.query('~(mall=="south.kr.ohou"|mall=="south.kr.himart"|mall=="south.kr.smartstore")').groupby(['rdate','cat_name','brand'], as_index=False)['final_sales_unit'].sum()

growth2 = growth2.sort_values(['cat_name','brand','rdate'],ascending=True)
growth2['sales_unit_lag'] = growth2['final_sales_unit'].shift(1)
growth2['cat_name_lag'] = growth2['cat_name'].shift(1)
growth2['brand_lag'] = growth2['brand'].shift(1)

growth2['total_ratio'] = np.where(
            (growth2['cat_name_lag'] == growth2['cat_name'])&
        (growth2['brand_lag'] == growth2['brand'])
        ,growth2['final_sales_unit']/growth2['sales_unit_lag']
        ,1)

growth2 = growth2.fillna(0)
growth2['total_ratio'] = np.where((growth2['sales_unit_lag']==0)&(growth2['final_sales_unit']!=0), growth2['final_sales_unit'], growth2['total_ratio'])
growth2['total_ratio'] = np.where(growth2['total_ratio'] == 0, 0, np.where(growth2['total_ratio'] < 0.28, 0.28, np.where(growth2['total_ratio'] > 3.5, 3.5, growth2['total_ratio'])))

growth2.head()

total_growth = growth2[['rdate','cat_name','brand','total_ratio']]

In [23]:
df7 = pd.merge(df6, total_growth, on=['rdate','cat_name','brand'],how='left')
df7 = pd.merge(df7, gmarket, on=['rdate','cat_name','brand'],how='left')
#df7 = pd.merge(df7, smartstore, on=['rdate','cat_name','brand'],how='left')
df7 = pd.merge(df7, ssgdotcom, on=['rdate','cat_name','brand'],how='left')
df7 = pd.merge(df7, eleven, on=['rdate','cat_name','brand'],how='left')
df7 = pd.merge(df7, auction, on=['rdate','cat_name','brand'],how='left')
#df7 = pd.merge(df7, tmon, on=['rdate','cat_name','brand'],how='left')
#df7 = pd.merge(df7, wemake, on=['rdate','cat_name','brand'],how='left')
df7 = df7.fillna(0)

# 데이터 예측

In [35]:
final = df7.query('mall=="south.kr.smartstore"')
final = final.query('(cat_name=="정수기"|cat_name=="음식물처리기")')

In [36]:
# 훈련,학습데이터 분류2
train = final.query('(rdate<"2025-12-28")')
test = final.query('(rdate>="2025-12-28")')

In [37]:
X_train = train.drop(['final_sales_unit','rdate','mall'], axis=1)
y_train = train[['final_sales_unit']]

X_test = test.drop(['final_sales_unit','rdate','mall'], axis=1)
y_test = test[['final_sales_unit']]

In [38]:
# 오리지널 인코딩 방식
s = (X_train.dtypes == 'object')
ns = (X_train.dtypes != 'object')
object_cols = list(s[s].index)
num_cols = list(ns[ns].index)
from sklearn.preprocessing import OrdinalEncoder

# 원본 데이터를 바꾸지 않기위해 복사본 생성
label_X_train = X_train.copy()
label_X_test = X_test.copy()

# 범주형 변수를 갖는 각 column에 ordinal encoder를 적용 
ordinal_encoder = OrdinalEncoder()
label_X_train[object_cols] = ordinal_encoder.fit_transform(X_train[object_cols])
label_X_test[object_cols] = ordinal_encoder.transform(X_test[object_cols])

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

label_X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
label_X_test[num_cols] = scaler.transform(X_test[num_cols])

In [39]:
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from scipy.stats import randint
from sklearn.metrics import make_scorer
# 85343
# 실제값과 예측값의 평균 오차를 성능 측정
def NMAE(true, pred):
    mae = np.sum(abs(true - pred))
    return mae

nmae_score = make_scorer(NMAE, greater_is_better=False)

# 최적 파라미터 탐색
param_grid = {
    'n_estimators': [200, 300, 400, 500, 600, 700],
    'depth': [2, 3, 4, 5, 6, 7],
    'learning_rate': [0.02, 0.04, 0.06, 0.08, 0.1],
    'min_child_samples': [10, 20, 30, 40],
    'grow_policy': ['SymmetricTree', 'Lossguide', 'Depthwise']
}


model_cat = CatBoostRegressor(random_state=42, eval_metric='R2')
model_cat.randomized_search(param_grid,
                            label_X_train, y_train,
                            cv=3, n_iter=10)

# 29743
model_cat.fit(label_X_train,y_train)
y_cat=model_cat.predict(label_X_test)
y_cat0 = y_cat.copy()
y_cat = np.where(y_cat<0,0,y_cat)
# 성능  42073
score_cat=NMAE(y_test, np.round(y_cat.reshape(-1,1),0))
print('score_cat_score : ',score_cat)

0:	learn: -0.4593365	test: -0.4982163	best: -0.4982163 (0)	total: 3.42ms	remaining: 1.71s
1:	learn: -0.3694477	test: -0.4045841	best: -0.4045841 (1)	total: 9.61ms	remaining: 2.39s
2:	learn: -0.2899095	test: -0.3238375	best: -0.3238375 (2)	total: 12.9ms	remaining: 2.14s
3:	learn: -0.2193567	test: -0.2536610	best: -0.2536610 (3)	total: 17.6ms	remaining: 2.18s
4:	learn: -0.1456526	test: -0.1725799	best: -0.1725799 (4)	total: 19.2ms	remaining: 1.9s
5:	learn: -0.0865508	test: -0.1122693	best: -0.1122693 (5)	total: 22.6ms	remaining: 1.86s
6:	learn: -0.0356972	test: -0.0602142	best: -0.0602142 (6)	total: 26ms	remaining: 1.83s
7:	learn: 0.0085990	test: -0.0134335	best: -0.0134335 (7)	total: 27.7ms	remaining: 1.7s
8:	learn: 0.0589357	test: 0.0412977	best: 0.0412977 (8)	total: 29.5ms	remaining: 1.61s
9:	learn: 0.0980917	test: 0.0787855	best: 0.0787855 (9)	total: 31.4ms	remaining: 1.54s
10:	learn: 0.1385923	test: 0.1225092	best: 0.1225092 (10)	total: 33.1ms	remaining: 1.47s
11:	learn: 0.1682719	t

93:	learn: 0.5662761	test: 0.4718272	best: 0.4718272 (93)	total: 199ms	remaining: 860ms
94:	learn: 0.5667539	test: 0.4716066	best: 0.4718272 (93)	total: 201ms	remaining: 858ms
95:	learn: 0.5690142	test: 0.4707756	best: 0.4718272 (93)	total: 204ms	remaining: 858ms
96:	learn: 0.5693817	test: 0.4708252	best: 0.4718272 (93)	total: 205ms	remaining: 852ms
97:	learn: 0.5702865	test: 0.4714949	best: 0.4718272 (93)	total: 206ms	remaining: 847ms
98:	learn: 0.5716299	test: 0.4718589	best: 0.4718589 (98)	total: 209ms	remaining: 846ms
99:	learn: 0.5727234	test: 0.4720692	best: 0.4720692 (99)	total: 212ms	remaining: 846ms
100:	learn: 0.5736220	test: 0.4721159	best: 0.4721159 (100)	total: 214ms	remaining: 846ms
101:	learn: 0.5753088	test: 0.4728808	best: 0.4728808 (101)	total: 215ms	remaining: 840ms
102:	learn: 0.5774496	test: 0.4717697	best: 0.4728808 (101)	total: 220ms	remaining: 849ms
103:	learn: 0.5782299	test: 0.4724072	best: 0.4728808 (101)	total: 223ms	remaining: 848ms
104:	learn: 0.5794209	te

249:	learn: 0.6936359	test: 0.4812478	best: 0.4840553 (213)	total: 591ms	remaining: 591ms
250:	learn: 0.6948501	test: 0.4802066	best: 0.4840553 (213)	total: 595ms	remaining: 590ms
251:	learn: 0.6952761	test: 0.4792289	best: 0.4840553 (213)	total: 599ms	remaining: 589ms
252:	learn: 0.6957848	test: 0.4793310	best: 0.4840553 (213)	total: 603ms	remaining: 589ms
253:	learn: 0.6964742	test: 0.4795676	best: 0.4840553 (213)	total: 606ms	remaining: 587ms
254:	learn: 0.6971320	test: 0.4796139	best: 0.4840553 (213)	total: 608ms	remaining: 584ms
255:	learn: 0.6980389	test: 0.4793315	best: 0.4840553 (213)	total: 610ms	remaining: 582ms
256:	learn: 0.6986044	test: 0.4793054	best: 0.4840553 (213)	total: 612ms	remaining: 579ms
257:	learn: 0.6992236	test: 0.4792895	best: 0.4840553 (213)	total: 615ms	remaining: 576ms
258:	learn: 0.6998250	test: 0.4791855	best: 0.4840553 (213)	total: 616ms	remaining: 573ms
259:	learn: 0.7003308	test: 0.4788404	best: 0.4840553 (213)	total: 618ms	remaining: 570ms
260:	learn

411:	learn: 0.7660227	test: 0.4650921	best: 0.4840553 (213)	total: 986ms	remaining: 211ms
412:	learn: 0.7663434	test: 0.4651231	best: 0.4840553 (213)	total: 988ms	remaining: 208ms
413:	learn: 0.7667276	test: 0.4651279	best: 0.4840553 (213)	total: 991ms	remaining: 206ms
414:	learn: 0.7669195	test: 0.4650239	best: 0.4840553 (213)	total: 994ms	remaining: 203ms
415:	learn: 0.7670660	test: 0.4650268	best: 0.4840553 (213)	total: 995ms	remaining: 201ms
416:	learn: 0.7672221	test: 0.4650801	best: 0.4840553 (213)	total: 999ms	remaining: 199ms
417:	learn: 0.7677949	test: 0.4652815	best: 0.4840553 (213)	total: 1000ms	remaining: 196ms
418:	learn: 0.7681544	test: 0.4652105	best: 0.4840553 (213)	total: 1s	remaining: 194ms
419:	learn: 0.7686014	test: 0.4650004	best: 0.4840553 (213)	total: 1s	remaining: 191ms
420:	learn: 0.7690844	test: 0.4648228	best: 0.4840553 (213)	total: 1s	remaining: 189ms
421:	learn: 0.7693445	test: 0.4649725	best: 0.4840553 (213)	total: 1.01s	remaining: 186ms
422:	learn: 0.7695

11:	learn: -0.2179225	test: -0.2544968	best: -0.2544968 (11)	total: 28.7ms	remaining: 1.17s
12:	learn: -0.1959043	test: -0.2330756	best: -0.2330756 (12)	total: 31.8ms	remaining: 1.19s
13:	learn: -0.1744477	test: -0.2105094	best: -0.2105094 (13)	total: 33.1ms	remaining: 1.15s
14:	learn: -0.1500910	test: -0.1841597	best: -0.1841597 (14)	total: 34.4ms	remaining: 1.11s
15:	learn: -0.1306489	test: -0.1638042	best: -0.1638042 (15)	total: 35.3ms	remaining: 1.07s
16:	learn: -0.1118621	test: -0.1445939	best: -0.1445939 (16)	total: 36.3ms	remaining: 1.03s
17:	learn: -0.0904720	test: -0.1220287	best: -0.1220287 (17)	total: 38.3ms	remaining: 1.02s
18:	learn: -0.0716206	test: -0.1032358	best: -0.1032358 (18)	total: 40.2ms	remaining: 1.02s
19:	learn: -0.0544210	test: -0.0849420	best: -0.0849420 (19)	total: 41.3ms	remaining: 991ms
20:	learn: -0.0346843	test: -0.0634826	best: -0.0634826 (20)	total: 42.6ms	remaining: 971ms
21:	learn: -0.0157220	test: -0.0428581	best: -0.0428581 (21)	total: 43.9ms	remai

...

225:	learn: 0.6850651	test: 0.4479248	best: 0.4482776 (199)	total: 390ms	remaining: 473ms
226:	learn: 0.6857985	test: 0.4480537	best: 0.4482776 (199)	total: 393ms	remaining: 472ms
227:	learn: 0.6862180	test: 0.4480366	best: 0.4482776 (199)	total: 395ms	remaining: 472ms
228:	learn: 0.6865429	test: 0.4474284	best: 0.4482776 (199)	total: 397ms	remaining: 469ms
229:	learn: 0.6871966	test: 0.4472347	best: 0.4482776 (199)	total: 398ms	remaining: 467ms
230:	learn: 0.6877755	test: 0.4472675	best: 0.4482776 (199)	total: 399ms	remaining: 465ms
231:	learn: 0.6884704	test: 0.4473911	best: 0.4482776 (199)	total: 404ms	remaining: 467ms
232:	learn: 0.6890333	test: 0.4468997	best: 0.4482776 (199)	total: 406ms	remaining: 465ms
233:	learn: 0.6894225	test: 0.4468888	best: 0.4482776 (199)	total: 407ms	remaining: 462ms
234:	learn: 0.6897286	test: 0.4476541	best: 0.4482776 (199)	total: 408ms	remaining: 460ms
235:	learn: 0.6903189	test: 0.4472474	best: 0.4482776 (199)	total: 409ms	remaining: 458ms
236:	learn

335:	learn: 0.7414144	test: 0.4497937	best: 0.4511129 (306)	total: 575ms	remaining: 281ms
336:	learn: 0.7419762	test: 0.4497785	best: 0.4511129 (306)	total: 576ms	remaining: 279ms
337:	learn: 0.7424226	test: 0.4498889	best: 0.4511129 (306)	total: 577ms	remaining: 277ms
338:	learn: 0.7426421	test: 0.4499253	best: 0.4511129 (306)	total: 579ms	remaining: 275ms
339:	learn: 0.7428428	test: 0.4499522	best: 0.4511129 (306)	total: 580ms	remaining: 273ms
340:	learn: 0.7431905	test: 0.4493235	best: 0.4511129 (306)	total: 581ms	remaining: 271ms
341:	learn: 0.7433581	test: 0.4493433	best: 0.4511129 (306)	total: 582ms	remaining: 269ms
342:	learn: 0.7437014	test: 0.4494468	best: 0.4511129 (306)	total: 584ms	remaining: 267ms
343:	learn: 0.7439345	test: 0.4495618	best: 0.4511129 (306)	total: 585ms	remaining: 265ms
344:	learn: 0.7444376	test: 0.4495754	best: 0.4511129 (306)	total: 592ms	remaining: 266ms
345:	learn: 0.7445786	test: 0.4495776	best: 0.4511129 (306)	total: 593ms	remaining: 264ms
346:	learn

444:	learn: 0.7852346	test: 0.4459823	best: 0.4511129 (306)	total: 759ms	remaining: 93.8ms
445:	learn: 0.7856045	test: 0.4460427	best: 0.4511129 (306)	total: 760ms	remaining: 92.1ms
446:	learn: 0.7857491	test: 0.4460746	best: 0.4511129 (306)	total: 762ms	remaining: 90.3ms
447:	learn: 0.7860827	test: 0.4462040	best: 0.4511129 (306)	total: 764ms	remaining: 88.7ms
448:	learn: 0.7862680	test: 0.4461724	best: 0.4511129 (306)	total: 767ms	remaining: 87.1ms
449:	learn: 0.7868192	test: 0.4464119	best: 0.4511129 (306)	total: 768ms	remaining: 85.3ms
450:	learn: 0.7871447	test: 0.4465332	best: 0.4511129 (306)	total: 769ms	remaining: 83.6ms
451:	learn: 0.7875503	test: 0.4468724	best: 0.4511129 (306)	total: 772ms	remaining: 82ms
452:	learn: 0.7878353	test: 0.4466944	best: 0.4511129 (306)	total: 773ms	remaining: 80.2ms
453:	learn: 0.7882014	test: 0.4464293	best: 0.4511129 (306)	total: 776ms	remaining: 78.6ms
454:	learn: 0.7885260	test: 0.4463602	best: 0.4511129 (306)	total: 778ms	remaining: 77ms
455

254:	learn: 0.6700429	total: 174ms	remaining: 167ms
255:	learn: 0.6707449	total: 174ms	remaining: 166ms
256:	learn: 0.6714345	total: 175ms	remaining: 166ms
257:	learn: 0.6721195	total: 176ms	remaining: 165ms
258:	learn: 0.6726886	total: 176ms	remaining: 164ms
259:	learn: 0.6728438	total: 177ms	remaining: 163ms
260:	learn: 0.6732508	total: 178ms	remaining: 163ms
261:	learn: 0.6738541	total: 178ms	remaining: 162ms
262:	learn: 0.6743430	total: 179ms	remaining: 161ms
263:	learn: 0.6746756	total: 180ms	remaining: 161ms
264:	learn: 0.6750274	total: 180ms	remaining: 160ms
265:	learn: 0.6759940	total: 181ms	remaining: 159ms
266:	learn: 0.6765652	total: 182ms	remaining: 159ms
267:	learn: 0.6771099	total: 182ms	remaining: 158ms
268:	learn: 0.6773782	total: 183ms	remaining: 157ms
269:	learn: 0.6777083	total: 184ms	remaining: 156ms
270:	learn: 0.6782257	total: 184ms	remaining: 156ms
271:	learn: 0.6789003	total: 185ms	remaining: 155ms
272:	learn: 0.6791459	total: 186ms	remaining: 154ms
273:	learn: 

dtype: float64


# 결과값

In [40]:
y_test.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

total = pd.concat([test[['rdate','cat_name','brand']],y_test,pd.DataFrame(np.round(y_cat,0).reshape(-1,1))], axis = 1)
#결과값 파일로 생성
total.rename(columns = {0:"predict_sales_unit"}, inplace = True)

In [41]:
total['final_sales_unit'].sum()

0

In [42]:
total['predict_sales_unit'].sum()

108.0

In [43]:
final = total.copy()
final = final.drop(['final_sales_unit'], axis=1)
final.rename(columns = {"predict_sales_unit":"sales_unit_y"}, inplace = True)
final.head(2)

,rdate,cat_name,brand,sales_unit_y
0,2025-12-28,음식물처리기,etc,57.0
1,2025-12-28,정수기,LG전자,0.0


In [34]:
# 저장
#final.to_csv("/home/jupyter/ECSM/3_lg/99_projection/projection/model/himart_1219tt.csv", index = False, encoding = "utf-8-sig")

save1 = final.copy()

save1.head()

,rdate,cat_name,brand,sales_unit_y
0,2025-12-28,TV,LG전자,34.0
1,2025-12-28,TV,etc,27.0
2,2025-12-28,TV,삼성전자,84.0
3,2025-12-28,공기청정기,LG전자,36.0
4,2025-12-28,공기청정기,etc,39.0


In [44]:
save2 = final.copy()
save2.head()

,rdate,cat_name,brand,sales_unit_y
0,2025-12-28,음식물처리기,etc,57.0
1,2025-12-28,정수기,LG전자,0.0
2,2025-12-28,정수기,etc,43.0
3,2025-12-28,정수기,삼성전자,8.0


In [45]:
save = pd.concat([save1, save2])

In [46]:
save.to_csv("/home/jupyter/ECSM/3_lg/01_Projection/projection/model/smart_1228.csv", index = False, encoding = "utf-8-sig")

In [ ]:
save.head()

In [ ]:
save.tail()

In [ ]:
# 변수 중요도
importances = pd.Series(model_cat.feature_importances_, label_X_train.columns)
import matplotlib.pyplot as plt

n = 20
plt.figure(figsize=(10,n/2))

plt.title(f'Ssgdotcom Important Features')
importances.sort_values()[-n:].plot.barh();

# 산출(분배)

In [47]:
save = pd.read_csv("/home/jupyter/ECSM/3_lg/01_Projection/projection/model/smart_1228.csv")

In [48]:
sql_host = "server5.marketingtool.co.kr"
sql_user = "direct"
sql_password = "Direct999"
sql_schema = "direct"
sql_port = 60001
sql_table = "lgcns_miecsm_daily_output_projection"

conn = pymysql.connect(host = sql_host,
                       user = sql_user,
                       passwd = sql_password,
                       db = sql_schema,
                       port = sql_port)
s = f'''mysql+pymysql://{sql_user}:{sql_password}@{sql_host}:{sql_port}/{sql_schema}'''
# , encoding='utf-8'
ENGINE = create_engine(s, encoding = "utf-8")

dm_user = "sales"
dm_password = "sales123"
dm_host = "server5.marketingtool.co.kr"
dm_port = 23061
dm_db = "lgcns_miecsm"
dm_uri = f'''mysql://{dm_user}:{dm_password}@{dm_host}:{dm_port}/{dm_db}'''

user = "direct"
password = "Direct999"
host = "server5.marketingtool.co.kr"
port = 60001
db = "direct"
uri = f'''mysql://{user}:{password}@{host}:{port}/{db}'''        
table = "lgcns_miecsm_daily_output"

In [49]:
target_date_list0 = pd.date_range("2025-12-28", "2025-12-28")

In [50]:
# 새로운 버전
for idx, target_date in enumerate(target_date_list0):
    func_round = lambda x: math.ceil(x) if (x % 1.0 == 0.5) else round(x)
    history_day = [0, 1, 2, 3]
    history_day_list = sorted(list(map(lambda x : (target_date - pd.Timedelta(days=x)).strftime("%Y-%m-%d"), history_day)))
    history_day_list_str = '","'.join(history_day_list)
    target_date = target_date.strftime("%Y-%m-%d")
    query = f"""
            SELECT * FROM {sql_schema}.{sql_table} 
            where 1=1
            and rdate in ("{history_day_list_str}")
            and mall = "south.kr.smartstore"
            """
    # output table 기준
    history_data = cx.read_sql(conn = uri, query = query)
    # 가격 구하기
    history_data2 = history_data[history_data['rdate']!=history_day_list[3]]
    price = history_data2.groupby('std_item_cd',as_index=False).agg({"final_sales_unit":"sum","final_sales_amount":"sum"})
    price['sale_price'] = price['final_sales_amount']/price['final_sales_unit']
    price = price.query('final_sales_unit!=0')
    price['sale_price'] = price['sale_price'].apply(func_round)
    ################################################################################################
    output_columns = list(set(history_data.columns) - set(['is_projection','final_sales_unit','final_sales_amount']))
    group_columns = list(set(history_data.columns) - set(['sales_amount','sales_unit','title','mall_opt_txt','hashcode','return_addr','mall_url','is_projection','final_sales_unit','final_sales_amount','mail_order_number','mail_order_name']))
    #master file
    dup_col = history_data.sort_values(['rdate'],ascending=True)
    dup_col = dup_col[['mall_pid','mall_opt_id','std_item_cd','retailer_number','product_hash','title','mall_opt_txt','hashcode','return_addr','mall_url','mail_order_number','mail_order_name']]
    dup_col = dup_col.drop_duplicates(subset= ['mall_pid','mall_opt_id','std_item_cd','retailer_number','product_hash'], keep="last")
    # 조인
    out1 = pd.merge(history_data,category_master, on = ["category"], how = "left")
    out2 = pd.merge(out1, item_master, on = ["std_item_cd"], how = "left")
    # 브랜드 구분
    out2['brand'] = np.where(out2['std_mnfr']=="LG전자", out2['std_mnfr'], np.where(out2['std_mnfr']=="삼성전자", out2['std_mnfr'],"etc"))
    
    # 가중치
    week3 = out2[out2['rdate']==history_day_list[0]]
    week3['sales_unit'] = week3['final_sales_unit']*0.2
    
    week2 = out2[out2['rdate']==history_day_list[1]]
    week2['sales_unit'] = week2['final_sales_unit']*0.3
    
    week1 = out2[out2['rdate']==history_day_list[2]]
    week1['sales_unit'] = week1['final_sales_unit']*0.5
    
    sum_total = pd.concat([week1,week2,week3], axis = 0)
    sum_total['rdate'] = target_date
    sum_total['rdate'] = pd.to_datetime(sum_total['rdate'])
    
    sum_total00 = sum_total.groupby(['cat_name','brand'],as_index=False)['sales_unit'].sum()
    
    # pf 구하기
    ev =  save[save["rdate"]==target_date]
    pf = pd.merge(sum_total00, ev[['cat_name','brand','sales_unit_y']],on=['cat_name','brand'], how='left')
    pf['ratio'] = pf['sales_unit_y']/pf['sales_unit']
    pf = pf.fillna(0)
    pf['ratio'] = np.where((pf['sales_unit']==0)&(pf['sales_unit_y']!=0), pf['sales_unit_y'], pf['ratio'])
    pf['ratio'] = np.where((pf['sales_unit_y']==0), 0.1, pf['ratio'])
    pf['ratio'] = np.where((pf['ratio']>=20), 20, pf['ratio'])
    
    # pf와 결합
    sum_total1 = pd.merge(sum_total, pf[['cat_name','brand','ratio']], on=['cat_name','brand'], how='left')
    sum_total1['sales_unit'] = sum_total1['sales_unit']*sum_total1['ratio']
    
    sum_total2 = sum_total1.groupby(group_columns,as_index=False)['sales_unit'].sum()
    sum_total2["sales_unit"] = sum_total2["sales_unit"].astype(float)
    sum_total2["sales_unit"] = sum_total2["sales_unit"].fillna(0)
    
    sum_total2['sales_unit'] = np.where((sum_total2['sales_unit']<1)&(sum_total2['sales_unit']>=0.25),1,sum_total2['sales_unit'].apply(func_round))
    
    
    sum_total3 = pd.merge(sum_total2, dup_col, on =['mall_pid','mall_opt_id','std_item_cd','retailer_number','product_hash'],how='left')
    sum_total4 = pd.merge(sum_total3, price[['std_item_cd','sale_price']], on="std_item_cd",how='left')
    sum_total4['sale_price'] = sum_total4['sale_price'].fillna(0)
    sum_total4['sales_amount'] = sum_total4['sale_price']*sum_total4['sales_unit']
    sum_total4 = sum_total4[output_columns]
    
    # 2회차 이상 구하기
    query1 = f"""
            SELECT rtime, mall_pid 
            FROM {dm_db}.po_dm_data_smartstore
            where 1=1
            and rdate in ("{target_date}")
            """
    query2 = f"""
            SELECT * FROM {db}.{table}
            where 1=1
            and rdate in ("{target_date}")
            and mall = "south.kr.smartstore"
            """
    
    # output table 기준
    group_size = cx.read_sql(conn = dm_uri, query = query1)
    group_size0 = group_size.drop_duplicates().groupby('mall_pid', as_index=False)['rtime'].nunique()
    
    today = cx.read_sql(conn = uri, query = query2)
    today = today[output_columns]
    # G마켓은 수집은 정상이라 해당 조건 변경
    today_data = pd.merge(today, group_size0, on="mall_pid", how='left').query('rtime>1')
    today_data.drop(columns = ["rtime"], inplace = True)

    cursor = conn.cursor()
    # 중복 mall_pid 제거
    mm = list(today_data['mall_pid'].unique())
    sum_total5 = sum_total4[(sum_total4['mall_pid'].isin(mm)==False)]
    #sum_total5 = sum_total4.copy()
    # 추정 여부 구분
    today_data['is_projection'] = 0
    sum_total5['is_projection'] = 1
    # 통합
    result = pd.concat([sum_total5, today_data], axis = 0)
    #result = sum_total5.copy()
    # 기존 Table과 merge
    group_columns_2 = list(set(history_data.columns) - set(['sales_amount','sales_unit','is_projection','final_sales_unit','final_sales_amount']))
    add_columns_2 = list(group_columns_2 + list(['is_projection','sales_unit','sales_amount']))
    output_columns_2 = list(set(history_data.columns))
    
    result_final = pd.merge(today, result[add_columns_2],on=group_columns_2, how="outer")
    
    result_final.rename(columns = {"sales_unit_x":"sales_unit","sales_amount_x":"sales_amount","sales_unit_y":"final_sales_unit","sales_amount_y":"final_sales_amount"}, inplace = True)
    result_final = result_final[output_columns_2]
    result_final['is_projection'] = result_final['is_projection'].fillna(1)
    result_final = result_final.fillna(0)
    
    query = f'''
    delete from {sql_schema}.{sql_table} where rdate = "{target_date}" and mall = "south.kr.smartstore"
    '''
    
    cursor.execute(query)
    conn.commit()
    
    result_final.to_sql(name = sql_table, con=ENGINE, if_exists='append', index = False,
                        chunksize = 10000) # 테이블에 INSERT


# 결과 검증

In [51]:
sogup_sdate = "2025-12-28"
sogup_edate = "2025-12-28"

In [52]:
import connectorx as cx
import pandas as pd
user = "direct"
password = "Direct999"
host = "server5.marketingtool.co.kr"
port = 60001
db = "direct"
table_list = ["lgcns_miecsm_daily_output"]
uri = f'''mysql://{user}:{password}@{host}:{port}/{db}'''
        
bucket = pd.DataFrame()
for table in table_list:
    query = f"""
    SELECT * FROM {db}.{table}
    where 1=1
    and rdate >= "{sogup_sdate}" and rdate <= "{sogup_edate}"
    """
    tmp = cx.read_sql(conn = uri, query = query)
    bucket = pd.concat([tmp, bucket], axis = 0)

In [53]:
import connectorx as cx
import pandas as pd
user = "direct"
password = "Direct999"
host = "server5.marketingtool.co.kr"
port = 60001
db = "direct"
table_list = ["lgcns_miecsm_daily_output_projection"]
uri = f'''mysql://{user}:{password}@{host}:{port}/{db}'''
bucket_re = pd.DataFrame()

for table in table_list:
    query = f"""
    SELECT * FROM {db}.{table}
    where 1=1
    and rdate >= "{sogup_sdate}" and rdate <= "{sogup_edate}"
    """
    tmp = cx.read_sql(conn = uri, query = query)
    bucket_re = pd.concat([tmp, bucket_re], axis = 0)

In [54]:
pd.set_option('display.max_rows',100)
bucket.query('mall=="south.kr.smartstore"').groupby('rdate',as_index=False)['sales_unit'].sum()

,rdate,sales_unit
0,2025-12-28,2102


In [55]:
bucket_re.query('mall=="south.kr.smartstore"').groupby('rdate',as_index=False)['sales_unit'].sum()

,rdate,sales_unit
0,2025-12-28,2102


In [56]:
bucket.query('mall=="south.kr.smartstore"').groupby('rdate',as_index=False)['final_sales_unit'].sum()

,rdate,final_sales_unit
0,2025-12-28,2102


In [57]:
bucket_re.query('mall=="south.kr.smartstore"').groupby('rdate',as_index=False)['final_sales_unit'].sum()

,rdate,final_sales_unit
0,2025-12-28,2325


In [58]:
bucket['final_sales_unit'].sum()

5093

In [59]:
bucket_re['final_sales_unit'].sum()

5316

In [ ]:
save = pd.read_csv("/home/jupyter/ECSM/3_lg/99_projection/projection/model/smart_0819.csv")